In [1]:
#Adding the libraries
import tensorflow as tf
import numpy as np
import os
import pandas as pd

In [2]:
#diresctory of the train and test dataset
train_dir = r"train"
test_dir = r"test"
os.path.exists(test_dir)

True

In [3]:
#reading the dataset which have all labels of all images
df = pd.read_csv('Training_set.csv')
df.shape

(6499, 2)

In [4]:
all_images_label = df['label'].unique()
all_images_label

<StringArray>
[         'SOUTHERN DOGFACE',                    'ADONIS',
            'BROWN SIPROETA',                   'MONARCH',
  'GREEN CELLED CATTLEHEART',           'CAIRNS BIRDWING',
      'EASTERN DAPPLE WHITE',               'RED POSTMAN',
          'MANGROVE SKIPPER',          'BLACK HAIRSTREAK',
             'CABBAGE WHITE',               'RED ADMIRAL',
              'PAINTED LADY',                'PAPER KITE',
                 'SOOTYWING',                'PINE WHITE',
                   'PEACOCK',        'CHECQUERED SKIPPER',
                     'JULIA',         'COMMON WOOD-NYMPH',
               'BLUE MORPHO',           'CLOUDED SULPHUR',
            'STRAITED QUEEN',            'ORANGE OAKLEAF',
           'PURPLISH COPPER',                     'ATALA',
           'IPHICLUS SISTER',             'DANAID EGGFLY',
              'LARGE MARBLE',          'PIPEVINE SWALLOW',
         'BLUE SPOTTED CROW',               'RED CRACKER',
             'QUESTION MARK',             

In [5]:
import shutil
def move_images(species):
    for images in df.loc[df['label'] == species, 'filename']:
        shutil.move(os.path.join(train_dir, images), os.path.join(os.path.join(train_v2_dir, species), images))

In [6]:
#for labels in uniq:
#    move_images(labels)

In [7]:
# from tensorflow.keras import mixed_precision
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_global_policy(policy)

In [8]:
#prepare the dataset

train_v2_dir = r"train_v2"
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=train_v2_dir,
    labels='inferred',
    validation_split=0.2,
    subset='training',
    batch_size=67,
    shuffle=True,
    color_mode='rgb',
    seed=123,
    image_size = (128,128)
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory=train_v2_dir,
    labels='inferred',
    validation_split=0.2,
    subset='validation',
    batch_size=67,
    shuffle=True,
    color_mode='rgb',
    seed=123,
    image_size = (128,128)
)
test_ds = tf.keras.utils.image_dataset_from_directory(
    directory=test_dir,
    labels=None,
    batch_size=67,
    shuffle=True,
    color_mode='rgb',
    image_size = (128,128)
)

Found 6499 files belonging to 75 classes.
Using 5200 files for training.
Found 6499 files belonging to 75 classes.
Using 1299 files for validation.
Found 2786 files.


In [9]:
#preprocessing the dataset
train_ds_final = (train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE))
val_ds_final = (val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE))
test_ds_final = (test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE))

In [10]:
#data augmentation and rescaling
aug = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomTranslation(0.2,0.2)
])
rescale_input = tf.keras.Sequential([
    tf.keras.layers.Resizing(100,100),
    tf.keras.layers.Rescaling(1./255)
])

In [11]:
#model deployment
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(shape=(128,128,3)),
    rescale_input,
    aug,
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu',padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(75,activation='softmax')
])

In [12]:
#custom optimizer
adam = tf.keras.optimizers.Adam(learning_rate=0.001,epsilon=1e-07)

In [16]:
model.compile(loss= tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=adam, metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_1 (Sequential)       │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 100, 100, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 50, 50, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 48, 48, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 22, 22, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 11, 11, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 75)             │         9,675 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,049,123 (4.00 MB)

 Trainable params: 349,707 (1.33 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 699,416 (2.67 MB)

In [18]:
history = model.fit(train_ds_final, validation_data=val_ds_final, epochs=200)

Epoch 1/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 13s 141ms/step - accuracy: 0.0177 - loss: 4.3190 - val_accuracy: 0.0231 - val_loss: 4.3166
Epoch 2/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 9s 117ms/step - accuracy: 0.0202 - loss: 4.3088 - val_accuracy: 0.0308 - val_loss: 4.2861
Epoch 3/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 12s 153ms/step - accuracy: 0.0256 - loss: 4.2221 - val_accuracy: 0.0423 - val_loss: 4.1132
Epoch 4/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 13s 162ms/step - accuracy: 0.0344 - loss: 4.0985 - val_accuracy: 0.0454 - val_loss: 4.0055
Epoch 5/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 12s 156ms/step - accuracy: 0.0392 - loss: 3.9751 - val_accuracy: 0.0362 - val_loss: 3.8730
Epoch 6/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 12s 150ms/step - accuracy: 0.0496 - loss: 3.8252 - val_accuracy: 0.0708 - val_loss: 3.6929
Epoch 7/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 16s 213ms/step - accuracy: 0.0588 - loss: 3.7381 - val_accuracy: 0.0662 - val_loss: 3.6455
Epoch 8/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 16s 201ms/step - accuracy: 0.0771 - loss: 3.6304 - v

KeyboardInterrupt: 